In [ ]:
""" 
Project: CSC 790
Team: 01
Authors: Rafail Islam, Yeboah Dacosta, Shashi Khanal
Project: Deep Convolutional Generative Adversarial Network for generating synthetic images of MNIST dataset
"""

In [1]:
# Imports library

import matplotlib.pyplot as plt
import numpy as np
from IPython import display
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model, Input
import os
import time


In [2]:
# Load & preprocess dataset

def load_data( buffer_size=60000, batch_size = 64):
    """ This function loads and preprocesses data, then return train dataset
    """
    # load dataset
    (train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
    
    # Reshape image into singel channel
    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
    
    # Normalize image
    # Normalize the images to [-1, 1]
    train_images = (train_images - 127.5) / 127.5 
    
    # slip data set by batch_size
    train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)
    
    return train_dataset

    

In [3]:
# Build Generator Model
def build_generator(noise_dim=100):
    """ Builds generator architecture and return the model
    """
    
    # input layer sahpe of noise_dim #Gaussian random numbers
    inputs = Input(shape=(noise_dim,) )
    
    dense_l1 = layers.Dense(256*7*7,activation='relu')(inputs) # start with low regulation image 7*7
    bn1 = layers.BatchNormalization() (dense_l1)
    lkr1 = layers.LeakyReLU()(bn1)
    # Reshape output
    rh1 = layers.Reshape((7, 7, 256))(lkr1)
    
    # Gradually upsampling
    # Conv2DTranspose layer 1
    cdt1 = layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same')(rh1)
    bn2 = layers.BatchNormalization() (cdt1)
    lkr2 = layers.LeakyReLU()(bn2)
    # Conv2DTranspose layer 2
    cdt2 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(lkr2)
    bn3 = layers.BatchNormalization() (cdt2)
    lkr3 = layers.LeakyReLU()(bn3)
    
    # Conv2DTranspose layer 3
    cdt3 = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(lkr3)
    
    
    model = Model(inputs=inputs,outputs= cdt3)
    
    return model
                    
    

In [4]:
model = build_generator(100)

In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 12544)             1266944   
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 128)         819328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)        

In [6]:
# Build Discriminator
def build_discriminator():
    """Builds discriminator architecture and returns model
    """
    
    inputs = Input(shape=(28,28,1,))
    
    convl1 = layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(inputs)
    lk1 = layers.LeakyReLU()(convl1)
    dp1 = layers.Dropout(0.3)(lk1)
    
    convl2 = layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(dp1)
    lk2 = layers.LeakyReLU()(convl2)
    dp2 = layers.Dropout(0.3)(lk2)


    ftl1 = layers.Flatten()(dp2)
    outputs = layers.Dense(1)(ftl1)
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [7]:
dis = build_discriminator()
dis.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)        

In [8]:
# 
def discriminator_loss(real_output, fake_output,cross_entropy):
    """ Calculates loss for discriminator model and returns it
    """
    # Loss for real images
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    # Loss for fake generated images
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    total_loss = real_loss + fake_loss
    
    return total_loss

In [9]:
def generator_loss(fake_output,cross_entropy):
    """ Calculates loss for generator model and returns it
    """
    gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    
    return gen_loss

In [10]:


@tf.function
def train(images,batch_size,noise_dim,generator,discriminator,generator_optimizer,discriminator_optimizer,cross_entropy):
    """ This function takes batch of train images and train and update both of the generator and discriminator model
    """
    # generating random noise 
    noise_vector = tf.random.normal([batch_size, noise_dim])
        
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        # gernerator model generate fake images
        gen_imgs = generator(noise_vector, training=True)
        
        # Discriminator classify real image and fake generated images
        real_outputs = discriminator(images, training=True)
        fake_outputs = discriminator(gen_imgs, training=True)
        
        # calculating loss ofr generator and dicriminator
        gen_loss = generator_loss(fake_outputs,cross_entropy)
        dis_loss = discriminator_loss(real_outputs, fake_outputs,cross_entropy)

    # get gradients
    gradient_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_dis = disc_tape.gradient(dis_loss, discriminator.trainable_variables)
    
    # apply gradients
    generator_optimizer.apply_gradients(zip(gradient_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradient_dis, discriminator.trainable_variables))
    
    return gen_loss, dis_loss*0.5

In [11]:
# Generate images with trained generator model
def generate_image(generator,sample_noise_vector,epoch):
    
    # generate images
    gen_imgs = generator(sample_noise_vector, training=False) # training = False to avoid updating weights

    fig = plt.figure(figsize=(4,4))

    for i in range(gen_imgs.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(gen_imgs[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig('./Project_1/images/image_at_epoch_{:04d}.png'.format(epoch+1))
    plt.show()

In [12]:
EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 16

buffer_size = 60000
batch_size = 200
noise_dim = 100

In [ ]:
# --- main block
generator = build_generator(100)

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

discriminator = build_discriminator()
img_label = discriminator(generated_image)
generator_optimizer = tf.keras.optimizers.Adam(0.001)
discriminator_optimizer = tf.keras.optimizers.Adam(0.001)

# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Random sample for generation fake image over the course of training model
sample_noise_vector = tf.random.normal([num_examples_to_generate, noise_dim])

train_dataset = load_data(buffer_size=buffer_size, batch_size=batch_size)

hist_gen_loss = []
hist_dis_loss = []
for epoch in range (EPOCHS):
    gen_loss = 0
    dis_loss = 0
    c = 0
    for batch_imgs in train_dataset:
        c +=1
        loss1, loss2 = train(batch_imgs,batch_size,noise_dim,generator,discriminator, generator_optimizer,discriminator_optimizer,cross_entropy)
        
        #print(loss1.numpy())
        #print(loss2.numpy())
        
        gen_loss += loss1.numpy()
        dis_loss += loss2.numpy()
        
        # generate few sample synthetic image while traing
        # and save it
    display.clear_output(wait=True)
        #print('Generator loss %.2f,\tDiscriminator loss: %.2f'%(loss1,loss2))
    if epoch % 5 == 0:
        generate_image(generator,sample_noise_vector,epoch)
    #generate_image(generator,sample_noise_vector,epoch)
        
    hist_gen_loss.append(gen_loss/c) # average loss per epoch
    hist_dis_loss.append(dis_loss/c)
        
    print('Epoch %d,\tGenerator loss %.2f,\tDiscriminator loss: %.2f'%(epoch+1,loss1,loss2))
            
        

Epoch 8,	Generator loss 2.88,	Discriminator loss: 0.50


In [ ]:
plt.plot(range(EPOCHS),hist_gen_loss,label='Generator loss')
plt.plot(range(EPOCHS),hist_dis_loss,label='Discriminator loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.savefig("Loss-vs-Epoch_DCGAN")
plt.show()


# Save model

In [ ]:
generator.save("./Project_1/SavedModel/GenModel1")

# Load Model

In [ ]:
saved_generator = tf.keras.models.load_model("./Project_1/SavedModel/GenModel1")

In [ ]:
sample_noise_vector = tf.random.normal([10, 100])
gen_imgs = saved_generator(sample_noise_vector, training=False) # training = False to avoid updating weights

fig = plt.figure(figsize=(10,10))

for i in range(gen_imgs.shape[0]):
    display.clear_output(wait=True)
    plt.imshow(gen_imgs[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
    plt.axis('off')
    plt.title("gen_image_"+str(i+1))

    plt.savefig('./Project_1/generated images/gen_image_{:05d}.png'.format(i+1))
    plt.show()